# Ranking evaluation metrics

In this lab you will implement ranking evaluation metrics.

For each query the search engine returns the sorted list of documents. We expect to have relevant documents at the top. 
In supervision learning to evaluate the quality we need labeled data.

Read this article first:
http://queirozf.com/entries/evaluation-metrics-for-ranking-problems-introduction-and-examples
https://medium.com/swlh/rank-aware-recsys-evaluation-metrics-5191bba16832

## Binary relevance metrics

We will start from the assumption, that each document is either relevant or not.

#### recall, precision, f1

First metrics are already faminiar for you from Introduction to Machine Learning course.
Implement relall, precision, and relevance for top $k$ documents.

`relevance` is a list which represents that document with this index in ranking is relevant or not.

In [0]:
import numpy as np
import math

def recall_at_k(relevance, k):
    return sum(relevance[:k]) / sum(relevance)

def precision_at_k(relevance, k):
    return sum(relevance[:k]) / k

def f1_at_k(relevance, k):
    p = precision_at_k(relevance, k)
    r = recall_at_k(relevance, k)
    return 2 * p * r / (p + r)

In [26]:
r = [1, 0, 1, 1, 0, 1, 0, 0]

print(recall_at_k(r, 1))
print(recall_at_k(r, 8))

print(precision_at_k(r, 1))
print(precision_at_k(r, 8))

print(f1_at_k(r, 1))
print(f1_at_k(r, 8))

0.25
1.0
1.0
0.5
0.4
0.6666666666666666


#### Average Precision
You can calculate the AP using the following algorithm:

<img src="http://queirozf.com/images/contents/mnc7sx1.png">

In [0]:
def average_precision(relevance, k):
    cor_pred = 0
    run_sum = 0
    relevance = [None] + relevance
    for i in range(1, k + 1):
        if relevance[i]:
            cor_pred += 1
            run_sum = run_sum + cor_pred / i
    return run_sum / sum(relevance[1:])

In [28]:
print(average_precision(r, 1))
print(average_precision(r, 8))

0.25
0.7708333333333333


## Relevance as a real number

#### DCG

DCG - discounted cumulative gain, does't require the relevance to be a binary feature. In many situations one document is more relevant than another and we want to represent it in supervised evaluation. Often the relevance is a number form ${0,1,2,3}$, but ${0,1}$ is also appropriate for usage.

The idea is that each rerelvant document brings a "gain" for a user. He or she looks through the documents from the first. So the gain sums cumulatively. But it is better to have a relevant document at the top of the ranking, so the weight of that gain decreases, or we have a discounded weight with increasing of the document position. And since the weight is decreasing, we can calculate this value for the top $k$ documents in ranking.

$$DCG@k = \sum_{1}^{k}\frac{2^{rel_i}-1}{log_2(i+1)}$$

In [0]:
from math import log2


def dcg_at_k(relevance, k=10):
    dcg = 0
    relevance = [None] + relevance
    for i in range(1, k + 1):
        dcg += (2 ** relevance[i] - 1) / log2(i + 1)
        # dcg += (relevance[i] ) / log2(i + 1) # alternative formula
    return dcg

In [30]:
r2 = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]

print(dcg_at_k(r, 1))
print(dcg_at_k(r, 8))

print(dcg_at_k(r2, 1))
print(dcg_at_k(r2, 10))

1.0
2.2868837451814152
7.0
16.80260104782745


#### nDCG
Now the idea is to normalize it to the maximum value.

In [0]:
def idgg_at_k(relevance, k=10):
    #relevance = [max(relevance[:k])] * len(relevance)
    return dcg_at_k(relevance, k)

def ndcg_at_k(relevance, k=10):
    dcg = dcg_at_k(relevance, k)
    idgg = idgg_at_k(sorted(relevance, reverse=True), k)
    return dcg / idgg

In [32]:
print(ndcg_at_k(r, 1))
print(ndcg_at_k(r, 8))

print(ndcg_at_k(r2, 1))
print(ndcg_at_k(r2, 10))


1.0
0.8927537907700456
1.0
0.8951337253357088


## Test in real data

You already have search engines on songs or news. Test the evaluation on real data.

1. Choose the search query query
2. Run the search
3. Manually look top 10 results and evaluate each of them if it relevant or not
4. Calculate AP and DCG (relevance is either 0 or 1)

In [33]:
# Your code here
"""❌✔️
Results for query: friend love
    id: 226 , Title: `Ain't Nobody Like My Baby`, Artists: `B.B. King`, Text: `I'm in love with a real f...`
    https://www.lyrics.com/lyric/3521789/B.B.+King/Ain%27t+Nobody+Like+My+Baby ❌

    id: 260 , Title: `World Weary Eyes`, Artists: `Arid`, Text: `You should have seen her...`
    https://www.lyrics.com/lyric/14954633/Arid/World+Weary+Eyes ✔️
    
    id: 132 , Title: `Sho' You Right`, Artists: `Barry White`, Text: `You know B.W. I've heard ...`
    https://www.lyrics.com/lyric/912845/Barry+White/Sho%27+You+Right ✔️
    
    id: 5 , Title: `More Than You Know`, Artists: `Billie Holiday`, Text: `Whether you are here or y...`
    https://www.lyrics.com/lyric/10671930/More+Than+You+Know ❌
    
    id: 135 , Title: `Don't Make Me Wait Too Long`, Artists: `Barry White`, Text: `Baby, it's really amazing...`
    https://www.lyrics.com/lyric/1638788/Barry+White/Don%27t+Make+Me+Wait+Too+Long ❌
    
    id: 421 , Title: `Sweet Pea`, Artists: `Tommy Roe`, Text: `I went to a dance just th...`
    https://www.lyrics.com/lyric/8528137/Tommy+Roe/Sweet+Pea ✔️
    
    id: 168 , Title: `Backyard [Single Version]`, Artists: `Salt-N-Pepa`, Text: `Do you get suspicious?...`
    https://www.lyrics.com/lyric/3522549/Various+Artists/Backyard+%5BSingle+Version%5D ✔️
    
    id: 360 , Title: `When A Man Loves A Woman`, Artists: `Percy Sledge`, Text: `When a man loves a woman...`
    https://www.lyrics.com/lyric/2005645/Percy+Sledge/When+a+Man+Loves+a+Woman ✔️
    
    id: 490 , Title: `Slow Down Jackson`, Artists: `Olivia Newton-John`, Text: `(Michel Brourman/Karen Go...`
    https://www.lyrics.com/lyric/23368415/Olivia+Newton-John/Slow+Down+Jackson ✔️
    
    id: 142 , Title: `Snakes`, Artists: `Papa Roach`, Text: `I got a problem with the ...`
    https://www.lyrics.com/lyric/3522735/Papa+Roach/Snakes ❌
"""

relevance = [0, 1, 1, 0, 0, 1, 1, 1, 1, 0]
print('AP', average_precision(relevance, k=len(relevance)))
print('DCG', dcg_at_k(relevance, k=len(relevance)))

AP 0.5882936507936508
DCG 2.436965146462523
